In [4]:
import os
import cv2
from ultralytics import YOLO
from datetime import datetime
import numpy as np

In [5]:
model = YOLO('yolov8m-seg.pt')

In [ ]:
import tqdm

test_images_dir = 'Livingroom'
output_dir = 'room_preds'  # To save annotated images in the same directory
laptop_output_dir = 'furniture_output'



if not os.path.exists(output_dir):
    os.makedirs(output_dir)
def resize_mask(mask, frame_shape):
    """
    Resize the mask to match the frame's shape.
    
    :param mask: The mask to resize.
    :param frame_shape: The shape of the frame.
    :return: Resized mask.
    """
    
    return cv2.resize(mask, (frame_shape[1], frame_shape[0]), interpolation=cv2.INTER_NEAREST)










def segment_and_annotate_images(batch_size=32):
    
    images = [img for img in os.listdir(test_images_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for i in tqdm.tqdm(range(0, len(images), batch_size)):
        batch_images = images[i:i+batch_size]
        img_paths = [os.path.join(test_images_dir, img_name) for img_name in batch_images]
        frames = [cv2.imread(img_path) for img_path in img_paths]


       
        preds = model.predict(img_paths, batch=batch_size)  # This should return a list of predictions


        for img_name, frame, pred in zip(batch_images, frames, preds):
            if img_name.startswith("annotated") or img_name.startswith("laptop"):
                continue
            img_path = os.path.join(test_images_dir, img_name)
            frame = cv2.imread(img_path)


            
            pred = model.predict(img_path)[0]  # Note -- Assuming this returns a list of predictions with masks
            frame_with_overlay = pred.plot()
            
            
            
            annotated_img_name = f"annotated_{img_name}"
            annotated_img_path = os.path.join(output_dir, annotated_img_name)
            cv2.imwrite(annotated_img_path, frame_with_overlay)
            
            #print(f"masks: {pred[0].masks}")


            #print(type(pred))
            #print(pred)
        
            
            ids = {56: "chair", 57: "couch", 58: "potted plant", 59: "bed", 60: "dining table", 62: "TV", 75: "vase"}
            laptopCount = 0
            laptop_class_id = 0
            for detection in pred:
                if detection.boxes and detection.masks:  # boxes and masks
                    for i, box in enumerate(detection.boxes):
                        #print(box.cls)
                        if box.cls.item() in ids:  
                            mask = detection.masks.data[i]  
                            resized_mask = resize_mask(mask.cpu().numpy(), frame.shape)


                            print(f"item: {box.cls.item()}")
                            mask_bool = resized_mask.astype(bool)


                            
                            rows = np.any(mask_bool, axis=1)
                            cols = np.any(mask_bool, axis=0)
                            rows_nonzero = np.where(rows)[0]
                            cols_nonzero = np.where(cols)[0]


                            if rows_nonzero.size and cols_nonzero.size:
                                y_min, y_max = rows_nonzero[[0, -1]]
                                x_min, x_max = cols_nonzero[[0, -1]]


                                bbox_area = (y_max - y_min + 1) * (x_max - x_min + 1)
                                # Calculate 
                                frame_area = frame.shape[0] * frame.shape[1]
                                if bbox_area / frame_area <= 0.01:
                                    continue


                                
                                cropped_frame = frame[y_min:y_max+1, x_min:x_max+1]
                                cropped_mask_bool = mask_bool[y_min:y_max+1, x_min:x_max+1]


                                
                                laptop_img_cropped = np.zeros_like(cropped_frame)


                                
                                laptop_img_cropped[cropped_mask_bool] = cropped_frame[cropped_mask_bool]
                            else:
                                continue


                            
                            laptop_img_name = f"{laptopCount}_{x_min}_{x_max}_{y_min}_{y_max}_{img_name}"
                            laptopCount+=1
                            laptop_img_path = f"{laptop_output_dir}/{laptop_img_name}"
                            path_with_png = laptop_img_path[0:len(laptop_img_path)-3] + "png"
                            cv2.imwrite(path_with_png, laptop_img_cropped)







# Process
segment_and_annotate_images()





  0%|          | 0/40 [00:00<?, ?it/s]


0: 640x640 1 bird, 4 chairs, 1 couch, 2 dining tables, 1 tv, 438.3ms
1: 640x640 1 couch, 1 potted plant, 3 vases, 438.3ms
2: 640x640 1 bottle, 1 couch, 2 potted plants, 438.3ms
3: 640x640 1 couch, 438.3ms
4: 640x640 1 couch, 1 potted plant, 438.3ms
5: 640x640 2 chairs, 1 couch, 2 potted plants, 1 dining table, 1 clock, 438.3ms
6: 640x640 1 couch, 438.3ms
7: 640x640 1 chair, 1 couch, 1 potted plant, 3 vases, 438.3ms
8: 640x640 2 chairs, 1 couch, 2 potted plants, 1 tv, 1 vase, 438.3ms
9: 640x640 4 chairs, 1 couch, 1 tv, 438.3ms
10: 640x640 1 chair, 1 couch, 3 potted plants, 1 dining table, 1 vase, 438.3ms
11: 640x640 1 bottle, 1 potted plant, 2 vases, 438.3ms
12: 640x640 1 bench, 2 chairs, 2 couchs, 1 bed, 438.3ms
13: 640x640 1 couch, 1 tv, 438.3ms
14: 640x640 2 chairs, 1 couch, 438.3ms
15: 640x640 1 couch, 438.3ms
16: 640x640 1 bowl, 4 chairs, 1 couch, 1 potted plant, 2 dining tables, 1 tv, 1 vase, 438.3ms
17: 640x640 4 chairs, 1 couch, 1 potted plant, 438.3ms
18: 640x640 3 couchs, 438

  2%|▎         | 1/40 [00:26<17:25, 26.81s/it]

item: 57.0
item: 56.0
item: 56.0
item: 58.0
item: 75.0
item: 57.0
item: 58.0
item: 56.0
item: 60.0

0: 640x640 4 chairs, 1 couch, 2 dining tables, 396.4ms
1: 640x640 1 couch, 1 dining table, 396.4ms
2: 640x640 2 chairs, 2 couchs, 1 potted plant, 1 vase, 396.4ms
3: 640x640 2 couchs, 396.4ms
4: 640x640 1 chair, 396.4ms
5: 640x640 2 chairs, 1 couch, 1 dining table, 396.4ms
6: 640x640 1 bowl, 2 chairs, 2 couchs, 2 potted plants, 1 dining table, 3 vases, 396.4ms
7: 640x640 1 bowl, 1 couch, 1 bed, 1 dining table, 1 tv, 396.4ms
8: 640x640 1 bowl, 3 chairs, 1 couch, 396.4ms
9: 640x640 3 chairs, 2 couchs, 2 potted plants, 4 vases, 396.4ms
10: 640x640 1 chair, 2 couchs, 2 potted plants, 1 book, 1 vase, 396.4ms
11: 640x640 2 chairs, 1 tv, 396.4ms
12: 640x640 1 chair, 1 couch, 2 potted plants, 1 vase, 396.4ms
13: 640x640 1 bowl, 3 chairs, 1 couch, 3 potted plants, 1 dining table, 1 tv, 2 vases, 396.4ms
14: 640x640 1 car, 1 chair, 1 couch, 4 clocks, 396.4ms
15: 640x640 1 bed, 396.4ms
16: 640x640 1 

  5%|▌         | 2/40 [00:52<16:31, 26.09s/it]

...

item: 57.0
item: 57.0
item: 57.0
item: 56.0
item: 58.0

0: 640x640 1 chair, 2 couchs, 2 potted plants, 1 tv, 1 clock, 3 vases, 385.6ms
1: 640x640 2 chairs, 3 couchs, 1 tv, 1 remote, 1 book, 385.6ms
2: 640x640 2 chairs, 1 couch, 2 potted plants, 1 vase, 385.6ms
3: 640x640 2 chairs, 1 couch, 1 refrigerator, 385.6ms
4: 640x640 2 couchs, 385.6ms
5: 640x640 2 suitcases, 1 bowl, 1 chair, 2 couchs, 2 potted plants, 1 book, 385.6ms
6: 640x640 2 couchs, 385.6ms
7: 640x640 2 chairs, 2 couchs, 2 books, 385.6ms
8: 640x640 3 chairs, 2 couchs, 3 potted plants, 1 vase, 385.6ms
9: 640x640 1 potted plant, 385.6ms
10: 640x640 1 chair, 385.6ms
11: 640x640 1 bowl, 2 chairs, 2 couchs, 1 potted plant, 1 tv, 3 vases, 385.6ms
12: 640x640 1 bottle, 2 chairs, 1 couch, 1 dining table, 385.6ms
13: 640x640 4 chairs, 2 couchs, 1 potted plant, 385.6ms
14: 640x640 1 couch, 1 book, 385.6ms
15: 640x640 3 chairs, 1 potted plant, 1 vase, 385.6ms
16: 640x640 3 chairs, 1 couch, 1 potted plant, 1 tv, 385.6ms
17: 640x640 2 c

 98%|█████████▊| 39/40 [16:46<00:24, 24.54s/it]

item: 57.0
item: 62.0
item: 56.0
item: 56.0
item: 58.0
item: 56.0

0: 640x640 3 couchs, 1 tv, 419.5ms
1: 640x640 2 couchs, 1 vase, 419.5ms
2: 640x640 1 chair, 2 couchs, 1 potted plant, 1 laptop, 419.5ms
3: 640x640 1 couch, 1 potted plant, 419.5ms
4: 640x640 5 chairs, 1 potted plant, 2 dining tables, 1 refrigerator, 419.5ms
5: 640x640 3 chairs, 1 couch, 1 potted plant, 419.5ms
6: 640x640 1 chair, 2 couchs, 1 potted plant, 1 dining table, 1 tv, 419.5ms
7: 640x640 2 cats, 2 chairs, 2 couchs, 4 potted plants, 1 tv, 1 vase, 419.5ms
8: 640x640 3 chairs, 1 couch, 2 potted plants, 419.5ms
9: 640x640 3 chairs, 1 couch, 419.5ms
10: 640x640 1 wine glass, 2 chairs, 1 couch, 1 potted plant, 419.5ms
11: 640x640 3 couchs, 1 book, 419.5ms
12: 640x640 2 chairs, 2 couchs, 1 tv, 419.5ms
13: 640x640 5 chairs, 2 couchs, 1 dining table, 419.5ms
14: 640x640 1 chair, 1 potted plant, 1 tv, 419.5ms
15: 640x640 1 bottle, 2 chairs, 1 couch, 419.5ms
16: 640x640 1 cup, 1 chair, 2 couchs, 1 potted plant, 1 dining ta

100%|██████████| 40/40 [17:06<00:00, 25.67s/it]

item: 57.0
item: 62.0
item: 57.0
item: 58.0


In [ ]:
import os

folder_path = "furniture_output"  


file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"Number of files in '{folder_path}': {file_count}")


Number of files in 'furniture_output': 4677


In [ ]:
import tqdm

test_images_dir = 'ebay_images'
output_dir = 'secondhand_preds'  
laptop_output_dir = 'secondhand_output'



if not os.path.exists(output_dir):
    os.makedirs(output_dir)
def resize_mask(mask, frame_shape):
    """
    Resize the mask to match the frame's shape.
    
    :param mask: The mask to resize.
    :param frame_shape: The shape of the frame.
    :return: Resized mask.
    """
    
    return cv2.resize(mask, (frame_shape[1], frame_shape[0]), interpolation=cv2.INTER_NEAREST)










def segment_and_annotate_images(batch_size=32):
    
    images = [img for img in os.listdir(test_images_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for i in tqdm.tqdm(range(0, len(images), batch_size)):
        batch_images = images[i:i+batch_size]
        img_paths = [os.path.join(test_images_dir, img_name) for img_name in batch_images]
        frames = [cv2.imread(img_path) for img_path in img_paths]
        preds = model.predict(img_paths, batch=batch_size)  
        for img_name, frame, pred in zip(batch_images, frames, preds):
            if img_name.startswith("annotated") or img_name.startswith("laptop"):
                continue
            img_path = os.path.join(test_images_dir, img_name)
            frame = cv2.imread(img_path)


            # Prediciting it
            pred = model.predict(img_path)[0]  
            frame_with_overlay = pred.plot()
            
            
            # Note -- Save
            annotated_img_name = f"annotated_{img_name}"
            annotated_img_path = os.path.join(output_dir, annotated_img_name)
            cv2.imwrite(annotated_img_path, frame_with_overlay)
            
            #print(f"masks: {pred[0].masks}")


            #print(type(pred))
            #print(pred)
        
        
            ids = {56: "chair", 57: "couch", 58: "potted plant", 59: "bed", 60: "dining table", 62: "TV", 75: "vase"}
            laptopCount = 0
            laptop_class_id = 0
            for detection in pred:
                if detection.boxes and detection.masks:  # Ensure there are boxes and masks
                    for i, box in enumerate(detection.boxes):
                        #print(box.cls)
                        if box.cls.item() in ids:  
                            mask = detection.masks.data[i]  # Getting the corresponding mask
                            resized_mask = resize_mask(mask.cpu().numpy(), frame.shape)
                            print(f"item: {box.cls.item()}")
                            mask_bool = resized_mask.astype(bool)
                            rows = np.any(mask_bool, axis=1)
                            cols = np.any(mask_bool, axis=0)
                            rows_nonzero = np.where(rows)[0]
                            cols_nonzero = np.where(cols)[0]
                            if rows_nonzero.size and cols_nonzero.size:
                                y_min, y_max = rows_nonzero[[0, -1]]
                                x_min, x_max = cols_nonzero[[0, -1]]
                                bbox_area = (y_max - y_min + 1) * (x_max - x_min + 1)
                                
                                frame_area = frame.shape[0] * frame.shape[1]
                                if bbox_area / frame_area <= 0.01:
                                    continue
                                cropped_frame = frame[y_min:y_max+1, x_min:x_max+1]
                                cropped_mask_bool = mask_bool[y_min:y_max+1, x_min:x_max+1]
                                
                                laptop_img_cropped = np.zeros_like(cropped_frame)
                                laptop_img_cropped[cropped_mask_bool] = cropped_frame[cropped_mask_bool]
                            else:
                                continue


                            
                            laptop_img_name = f"{laptopCount}_{x_min}_{x_max}_{y_min}_{y_max}_{img_name}"
                            laptopCount+=1
                            laptop_img_path = f"{laptop_output_dir}/{laptop_img_name}"
                            path_with_png = laptop_img_path[0:len(laptop_img_path)-3] + "png"
                            cv2.imwrite(path_with_png, laptop_img_cropped)
# Execute the function






# Process the images
segment_and_annotate_images()





  0%|          | 0/5 [00:00<?, ?it/s]


0: 640x640 1 person, 1 stop sign, 1 bench, 1 kite, 1 chair, 438.0ms
1: 640x640 1 potted plant, 438.0ms
2: 640x640 1 bench, 438.0ms
3: 640x640 1 bench, 438.0ms
4: 640x640 2 potted plants, 1 dining table, 3 vases, 438.0ms
5: 640x640 1 chair, 438.0ms
6: 640x640 4 chairs, 438.0ms
7: 640x640 3 chairs, 438.0ms
8: 640x640 1 couch, 1 bed, 438.0ms
9: 640x640 (no detections), 438.0ms
10: 640x640 (no detections), 438.0ms
11: 640x640 6 chairs, 438.0ms
12: 640x640 1 bed, 438.0ms
13: 640x640 2 couchs, 438.0ms
14: 640x640 1 chair, 1 dining table, 1 book, 438.0ms
15: 640x640 1 suitcase, 1 oven, 438.0ms
16: 640x640 4 chairs, 1 dining table, 1 remote, 438.0ms
17: 640x640 1 bicycle, 1 chair, 1 potted plant, 1 dining table, 438.0ms
18: 640x640 1 bench, 438.0ms
19: 640x640 1 suitcase, 438.0ms
20: 640x640 (no detections), 438.0ms
21: 640x640 1 apple, 1 potted plant, 438.0ms
22: 640x640 2 chairs, 438.0ms
23: 640x640 (no detections), 438.0ms
24: 640x640 1 bench, 438.0ms
25: 640x640 1 chair, 1 refrigerator, 4

 20%|██        | 1/5 [00:25<01:40, 25.02s/it]

item: 59.0

0: 640x640 2 chairs, 361.3ms
1: 640x640 1 chair, 2 couchs, 1 tv, 1 vase, 361.3ms
2: 640x640 (no detections), 361.3ms
3: 640x640 1 bottle, 1 dining table, 361.3ms
4: 640x640 2 bottles, 2 bowls, 1 vase, 361.3ms
5: 640x640 1 bed, 361.3ms
6: 640x640 1 bench, 361.3ms
7: 640x640 2 chairs, 361.3ms
8: 640x640 2 clocks, 361.3ms
9: 640x640 1 bench, 361.3ms
10: 640x640 (no detections), 361.3ms
11: 640x640 (no detections), 361.3ms
12: 640x640 (no detections), 361.3ms
13: 640x640 (no detections), 361.3ms
14: 640x640 1 clock, 361.3ms
15: 640x640 1 chair, 361.3ms
16: 640x640 1 bed, 361.3ms
17: 640x640 (no detections), 361.3ms
18: 640x640 1 oven, 1 refrigerator, 361.3ms
19: 640x640 1 bench, 361.3ms
20: 640x640 1 person, 1 clock, 361.3ms
21: 640x640 (no detections), 361.3ms
22: 640x640 1 person, 1 bench, 361.3ms
23: 640x640 2 bananas, 361.3ms
24: 640x640 2 chairs, 361.3ms
25: 640x640 1 bed, 361.3ms
26: 640x640 1 bench, 3 chairs, 1 dining table, 361.3ms
27: 640x640 (no detections), 361.3ms
2

 40%|████      | 2/5 [00:46<01:08, 22.90s/it]

...


0: 640x640 1 bird, 377.1ms
1: 640x640 1 chair, 1 couch, 1 bed, 377.1ms
2: 640x640 1 clock, 377.1ms
3: 640x640 (no detections), 377.1ms
4: 640x640 1 bench, 377.1ms
5: 640x640 1 sports ball, 1 bed, 377.1ms
6: 640x640 1 toilet, 377.1ms
7: 640x640 2 couchs, 377.1ms
8: 640x640 6 chairs, 377.1ms
9: 640x640 1 cup, 1 chair, 1 dining table, 377.1ms
10: 640x640 1 chair, 1 bed, 377.1ms
11: 640x640 1 chair, 1 laptop, 377.1ms
12: 640x640 1 bench, 377.1ms
13: 640x640 1 bed, 377.1ms
14: 640x640 2 chairs, 377.1ms
15: 640x640 1 suitcase, 377.1ms
16: 640x640 3 chairs, 1 dining table, 377.1ms
17: 640x640 (no detections), 377.1ms
18: 640x640 (no detections), 377.1ms
19: 640x640 1 potted plant, 1 vase, 377.1ms
20: 640x640 1 person, 1 bench, 1 chair, 377.1ms
21: 640x640 (no detections), 377.1ms
22: 640x640 (no detections), 377.1ms
23: 640x640 1 oven, 377.1ms
24: 640x640 10 chairs, 377.1ms
25: 640x640 1 chair, 377.1ms
26: 640x640 1 couch, 377.1ms
27: 640x640 1 scissors, 377.1ms
28: 640x640 1 boat, 1 surfboa

 80%|████████  | 4/5 [01:32<00:22, 22.96s/it]


0: 640x640 1 chair, 341.0ms
1: 640x640 1 sports ball, 341.0ms
2: 640x640 (no detections), 341.0ms
3: 640x640 1 couch, 341.0ms
4: 640x640 1 chair, 1 couch, 341.0ms
5: 640x640 1 bed, 341.0ms
Speed: 2.6ms preprocess, 341.0ms inference, 1.8ms postprocess per image at shape (6, 3, 640, 640)

image 1/1 c:\Users\vsrik\Python\Hack_TJ\ebay_images\image_2_65.png: 640x352 1 chair, 204.7ms
Speed: 1.1ms preprocess, 204.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 352)
item: 56.0

image 1/1 c:\Users\vsrik\Python\Hack_TJ\ebay_images\image_2_66.png: 640x448 1 sports ball, 1 cake, 231.4ms
Speed: 1.6ms preprocess, 231.4ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 c:\Users\vsrik\Python\Hack_TJ\ebay_images\image_2_67.png: 480x640 (no detections), 237.8ms
Speed: 1.8ms preprocess, 237.8ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\vsrik\Python\Hack_TJ\ebay_images\image_2_7.png: 384x640 (no detections), 238.0ms
Sp

100%|██████████| 5/5 [01:36<00:00, 19.20s/it]

item: 59.0
